In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm.notebook import tqdm, trange
import pickle

In [2]:
headers = {
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Methods": "GET",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Max-Age": "3600",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
}

In [3]:
store_url = "https://restaurants.subway.com/united-states"
loc_url = "https://restaurants.subway.com/"

In [4]:
link_dic = {}

In [5]:
page = requests.get(store_url, headers)
soup = BeautifulSoup(page.content, "html.parser")
loclist = soup.find(class_="Directory-listLinks")

In [ ]:
state_html = []
for li in loclist.find_all("li"):
    a = li.find(class_="Directory-listLink")
    if a["href"][-2:] != 'ns':
        link_dic[a["href"][-2:]] = []
        state_html.append(loc_url+a["href"])

In [9]:
print(link_dic)


{'al': [], 'ak': [], 'az': [], 'ar': [], 'ca': [], 'co': [], 'ct': [], 'de': [], 'fl': [], 'ga': [], 'hi': [], 'id': [], 'il': [], 'in': [], 'ia': [], 'ks': [], 'ky': [], 'la': [], 'me': [], 'md': [], 'ma': [], 'mi': [], 'mn': [], 'ms': [], 'mo': [], 'mt': [], 'ne': [], 'nv': [], 'nh': [], 'nj': [], 'nm': [], 'ny': [], 'nc': [], 'nd': [], 'oh': [], 'ok': [], 'or': [], 'pa': [], 'ri': [], 'sc': [], 'sd': [], 'tn': [], 'tx': [], 'ut': [], 'vt': [], 'va': [], 'wa': [], 'dc': [], 'wv': [], 'wi': [], 'wy': []}


In [ ]:
print(len(state_html))
for i in state_html:
    print(i)

In [20]:
store_href = []
loop = tqdm(state_html)

for state in loop:
    loop.set_description(f"working on {state}")

    st_page = requests.get(state, headers)
    st_soup = BeautifulSoup(st_page.content, "html.parser")
    st_list = st_soup.find(class_="Directory-listLinks")

    for links in st_list.find_all("li"):
        g = links.find(class_="Directory-listLink")
        store_href.append(g["href"][3:])
    sleep(0.5)

  0%|          | 0/51 [00:00<?, ?it/s]

In [27]:
print(store_href[1:10])

['united-states/al/adamsville', 'united-states/al/alabaster', 'united-states/al/albertville/7343-highway-431', 'united-states/al/alexander-city/875-market-place', 'united-states/al/alexandria/intersec-of-us-431-&-gladden', 'united-states/al/aliceville/300-2nd-street-nw', 'united-states/al/andalusia', 'united-states/al/anniston', 'united-states/al/arab']


In [ ]:
city_list = []
for i in store_href:
    count = i.count('/')
    if count == 2:
        city_list.append(i)
    elif count == 3:
        link_dic[i[14:16]].append(loc_url+i)
print(city_list)


In [ ]:
for city,n in zip(city_list,range(len(city_list))):
    city_list[n] = loc_url+city
print(city_list)

In [37]:
final_list = []
loop1 = tqdm(city_list)

for city_store in loop1:
    loop1.set_description(f"working on {city_store}")

    city_page = requests.get(city_store, headers)
    city_soup = BeautifulSoup(city_page.content, "html.parser")
    cities = city_soup.find_all(class_="Teaser-title")

    for links in cities:
        final_list.append(links["href"][6:])
    sleep(0.28)

  0%|          | 0/3362 [00:00<?, ?it/s]

In [43]:
print(final_list[1])

united-states/al/adamsville/4024-veterans-memorial-hwy


In [44]:
for i in final_list:
    link_dic[i[14:16]].append(loc_url+i)

In [ ]:
print(link_dic)

In [46]:
key_to_value_lengths = {k:len(v) for k, v in link_dic.items()}
print(sum([len(x) for x in link_dic.values()]))
print(key_to_value_lengths)

22933
{'al': 415, 'ak': 62, 'az': 411, 'ar': 263, 'ca': 2310, 'co': 395, 'ct': 300, 'de': 24, 'fl': 1345, 'ga': 773, 'hi': 105, 'id': 124, 'il': 1037, 'in': 597, 'ia': 288, 'ks': 235, 'ky': 389, 'la': 450, 'me': 108, 'md': 435, 'ma': 328, 'mi': 844, 'mn': 426, 'ms': 296, 'mo': 494, 'mt': 80, 'ne': 190, 'nv': 198, 'nh': 90, 'nj': 216, 'nm': 164, 'ny': 902, 'nc': 769, 'nd': 68, 'oh': 1057, 'ok': 349, 'or': 276, 'pa': 753, 'ri': 68, 'sc': 370, 'sd': 94, 'tn': 582, 'tx': 1987, 'ut': 198, 'vt': 46, 'va': 636, 'wa': 545, 'dc': 62, 'wv': 178, 'wi': 541, 'wy': 60}


In [ ]:
print(link_dic["nm"])

In [145]:
with open('/Users/benperry/Documents/Python/FastFood/LinkDicts/subway.pkl', 'wb') as f:
    pickle.dump(link_dic, f)